In [1]:
from typing import (
    List,
    Dict
)
from itertools import product
from functools import partial
from pdb import set_trace

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import pairwise_kernels

import gc4eptn
from gc4eptn import dataloaders
from gc4eptn.utils import norms
from gc4eptn.ggm import single_ggm_exp
from gc4eptn.utils.utils import (
    get_dataloader,
    to_adjacency,
    build_experiment_path,
    filter_function_kwargs,
    lambda_grid_R_max_min,
    lambda_grid_R_max_square,
    exp_log_grid,
)
from gc4eptn.utils.plotting import plot_eptn_ground_truths, R_mag_ang_plots
from gc4eptn.utils.kernels import (
    weighted_kernel,
    corr,
    cov,
    gram,
    rational_quadratic,
)
from gc4eptn.utils.metrics import (
    graph_fscore,
    graph_fbscore,
    graph_precision,
    graph_recall,
    hamming_distance,
    ebic,
)
np.set_printoptions(suppress=True)
%load_ext autoreload
%autoreload 2

In [2]:
rqk = partial(rational_quadratic, length_scale=50, alpha=1.0)
rbf = partial(pairwise_kernels, metric='rbf')

wcorr = partial(weighted_kernel, kernel=corr)
wcov = partial(weighted_kernel, kernel=cov)
wgram = partial(weighted_kernel, kernel=partial(gram, transpose=True))
wrq = partial(weighted_kernel, kernel=rqk)
wrbf = partial(weighted_kernel, kernel=rbf)

In [3]:
fstandardize = partial(
    norms.feature_norm,
    norm_fn=norms.standardize,
    features=['V_mag', 'V_ang', 'I_ang', 'I_mag']
)

In [4]:
metrics = {
    'hd': dict(
        fn=hamming_distance,
        type='lowest'
    ),
    'ebic': dict(
        fn=partial(ebic, gamma=0.5), 
        type='lowest'
    ),
    'f-score': dict(
        fn=graph_fscore,
        type='highest'
    ),
    'fb-score': dict(
        fn=partial(graph_fbscore, beta=2),
        type='highest'
    ),
    'precision': dict(
        fn=graph_precision,
        type='highest'
    ),
    'recall': dict(
        fn=graph_recall,
        type='highest'
    ),
}

lambda_grid=exp_log_grid(start=0.001, end=200, n=100, round_dec=4)
ggmncv_kwargs = {
    'penalize_diagonal': False,
    'LLA': False,
    'select': 'lambda',
    'lambda': lambda_grid, 
    'ic': 'ebic'
}

# Multi-Experiment Runner

In [5]:
def multi_ggm_exps(
    dataset_type: str = 'rtds',
    dataset_versions: List[str] = ['v5'],
    difficulties: List[str] = ['novice'],
    topologies = ['complete', 'partial-left', 'partial-right'],
    load_types: List[str] = ['high', 'medium', 'low'],
    n_samples: int = [None],
    metrics: Dict = metrics,
    best_metric_names: List[str] = ['fb-score'], 
    net_pmu_kwargs: List[Dict] = [None],
    norm_types: List[str] = ['fstandardize'],
    seed: int = 0,
    verbose: bool = False,
    *,
    R_types: List[str] = ['wrq', 'rq', 'wcorr'],
    penalties: List = ['lasso', 'scad', 'mcp', 'atan', 'selo', 'exp'],
):
    """ Automates running of multiple GGM experiements. 

        Args:
            norm_types: List containing the names of normalization functions to run.
                Use the string 'raw' to apply NO normalization.
    """
    pmu = list(product(dataset_versions, difficulties, topologies, load_types, net_pmu_kwargs))
    build = list(product(norm_types, n_samples, R_types))
    exps = list(product(best_metric_names, penalties))
    ########################################################################
    for dataset_version, difficulty, topology, load, net_kwargs in tqdm(pmu):
        pmuds_class = get_dataloader(dataset_type, dataset_version, difficulty)

        net_kwargs = net_kwargs if net_kwargs is not None else {}
        net_kwargs['load'] = load
        net_kwargs['topology'] = topology
        
        net_pmuds = pmuds_class(**net_kwargs).load_data()

        data_type_path = build_experiment_path(
            subdirs=[
                dataset_type,
                dataset_version,
                'ggm',
                'multi-exp',
                difficulty,
                topology,
                'simple' if net_pmuds.drop_parallel_currents else 'multi-edge',
                '-'.join(load) if isinstance(load, list) else load,
            ]
        ) 
       #########################################
        for norm_type, n, R_type in tqdm(build):
            norm_fn = globals()[norm_type] if  norm_type is not None else None
            X = net_pmuds.build_graph_data(
                n=n, 
                norm_fn=norm_fn,
                random=True,
                rng=np.random.default_rng(seed=seed),
            ).graph_df
            n = n if n is not None else len(net_pmuds.df)
            R = globals()[R_type](X)
            
            save_dir_path = build_experiment_path(
                subdirs=[
                    norm_type, 
                    n,
                    R_type,
                ],
                path=data_type_path,
            )
            print(f"Save dir path: {save_dir_path}")
            ########################################
            for select_best_metric_name, penalty in exps:
                select_best_metric = metrics[select_best_metric_name]
                save_path_score = save_dir_path/select_best_metric_name

                single_ggm_exp(
                    R=R,
                    n_samples=n,
                    penalty=penalty,
                    A=net_pmuds.true_network_graph,
                    positions=net_pmuds.graph_positions,
                    metric_fn=select_best_metric['fn'],
                    select_metric=select_best_metric['type'],
                    tau=None,
                    labels=net_pmuds.labels,
                    annot=True,
                    save_dir_path=save_path_score,
                    file_prefix=f"{norm_type}-{R_type}-{select_best_metric_name}",
                    ggmncv_kwargs=ggmncv_kwargs,
                    verbose=verbose,
                    disable_plotting=True
                )
                # prevent plots from being stored past saving
                plt.clf()
                plt.close('all')
            

Notes:
- When dropping parallel edges (i.e., currents) and using partial-left or partial-right the graphs will be the same as complete, except the values will differ. This is because `drop_parallel_currents` drops the already disconnected branch. 

In [37]:
multi_ggm_exps(
    dataset_type='rtds',
    dataset_versions=['v5'],
    difficulties=['novice'],
    R_types=['rqk'],
    penalties = ['lasso', 'scad', 'mcp', 'atan', 'selo'],
    net_pmu_kwargs=[dict(drop_parallel_currents=True)]
)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/rtds/v5/ggm/multi-exp/novice/complete/simple/high/fstandardize/1802/rqk


R[write to console]: selecting lambda



  |======================================================================| 100%

...

  |======================================================================| 100%

R[write to console]: selecting lambda



  |======================================================================| 100%

R[write to console]: selecting lambda



  |======================================================================| 100%

## Case9

In [39]:
multi_ggm_exps(
    dataset_type='matpower',
    dataset_versions=['case9'],
    difficulties=['novice'],
    topologies = ['complete'],
    load_types=['80-90', '100-110', '110-120', '80-120'],
    R_types=['rqk'],
    penalties = ['lasso', 'scad', 'mcp', 'atan', 'selo'],
)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/matpower/case9/ggm/multi-exp/novice/complete/simple/80-90/fstandardize/5000/rqk


R[write to console]: selecting lambda



  |======================================================================| 100%

...

  |======================================================================| 100%

R[write to console]: selecting lambda



  |======================================================================| 100%

R[write to console]: selecting lambda



  |======================================================================| 100%

## Case14

In [ ]:
multi_ggm_exps(
    dataset_type='matpower',
    dataset_versions=['case14'],
    difficulties=['novice'],
    topologies = ['complete'],
    load_types=['80-90', '100-110', '110-120', '80-120'],
    R_types=['rqk'],
    penalties = ['lasso', 'scad', 'mcp', 'atan', 'selo'],
)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/matpower/case14/ggm/multi-exp/novice/complete/simple/80-90/fstandardize/5000/rqk


R[write to console]: selecting lambda



  |======================================================================| 100%

...

  |================================                                      |  46%

Exception ignored from cffi callback <function _consolewrite_ex at 0x7ff5a34e13f0>:
Traceback (most recent call last):
  File "/home/ben/miniconda3/envs/gc4eptn/lib/python3.10/site-packages/rpy2/rinterface_lib/callbacks.py", line 130, in _consolewrite_ex
    @ffi_proxy.callback(ffi_proxy._consolewrite_ex_def,
KeyboardInterrupt: 


  |===================================                                   |  50%

Exception ignored from cffi callback <function _consolewrite_ex at 0x7ff5a34e13f0>:
Traceback (most recent call last):
  File "/home/ben/miniconda3/envs/gc4eptn/lib/python3.10/site-packages/rpy2/rinterface_lib/callbacks.py", line 130, in _consolewrite_ex
    @ffi_proxy.callback(ffi_proxy._consolewrite_ex_def,
KeyboardInterrupt: 


  |====================================                                  |  51%